In [10]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array


cap = cv2.VideoCapture('dia1.webm')
car_C = 0
#SF
SF = cv2.bgsegm.createBackgroundSubtractorMOG()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

#auto_model = load_model('Modeloscheck')
auto_model = load_model('Cars.model7P2P')
auto_model.load_weights('my_weights7P2P')

#CUERPO
while True:
    ret, frame = cap.read()
    if ret == False: break
        
#METOD        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   #GRAYIMAG
    cv2.rectangle(frame, (0,0), (frame.shape[1],40), (0,0,0), -1) #TITLE MOV
    color = (0,255,0)
    Text = 'No Movimiento parqueadero'
    
#AREAS
    area_park = np.array([[130,270], [840,170], [1050,220], [50,480]])
    area_entrada = np.array([[1278,580], [1278,380], [1150,350], [1050,480]])
    
#AREA BLANCA    
    Imx = np.zeros(shape=(frame.shape[:2]),dtype=np.uint8) #area blanca parki
    ImxC = np.zeros(shape=(frame.shape[:2]),dtype=np.uint8) #area blanca cont
#COUNTOURS
    Imx = cv2.drawContours(Imx, [area_park], -1,(255),-1) # area parking white
    ImxC = cv2.drawContours(ImxC, [area_entrada], -1,(255),-1) # area parking white
#ESCALAS DE GRISES
    ImxF= cv2.bitwise_and(gray, gray, mask=Imx) #Escala de grises solo area parking
    ImxCC= cv2.bitwise_and(gray, gray, mask=ImxC) #Escala de grises solo area parking
    
#SUBSTRACTO   
    SKmask = SF.apply(ImxF) 
    SKmask_C = SF.apply(ImxCC) 
#MORPHO   
    SKmask = cv2.morphologyEx(SKmask, cv2.MORPH_OPEN, kernel) #MAYOR DENSIDAD PIXEL
    SKmask_C = cv2.morphologyEx(SKmask_C, cv2.MORPH_OPEN, kernel) #MAYOR DENSIDAD PIXEL
#DILATACION   
    SKmask = cv2.dilate(SKmask, None, iterations=5) #ILT une areas alejadas
    SKmask_C = cv2.dilate(SKmask_C, None, iterations=4) #ILT une areas alejadas  4
    
#ENCONTRAR CONTORNOS   
    cont = cv2.findContours(SKmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    cont_C = cv2.findContours(SKmask_C, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
#PARKING
    for conts in cont:
        if cv2.contourArea(conts) > 100:  #100
            x,y,w,h = cv2.boundingRect(conts)
            cv2.rectangle(frame,(x,y),(x+w,y+h), (0,255,0),2)
            Text = 'Movimiento en parqueadero'
            color = (0,0,255)
#ENTRADA    
    autos_list=[]
    predictions=[]
    for conts_c in cont_C:
        if cv2.contourArea(conts_c) > 1200:  #1200
            x,y,w,h = cv2.boundingRect(conts_c)
            cv2.rectangle(frame,(x,y),(x+w,y+h), (0,255,0),2)
            roi = frame[y:y+h, x:x+w]
            roi = cv2.resize(roi, (100,100))
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            roi = roi/255.0
            autos_list.append(roi)
            #Model 
            if len(autos_list)>0:
                for i in autos_list:
                    predictions = auto_model.predict(i)
                for pred in predictions:
                    respuesta = np.argmax(pred)
                    #print(predictions)    
                    if respuesta == 0:
                        titleobj = 'Camioneta'
                        cv2.putText(frame, titleobj, (x,y-10),1,0.7,(0,255,0),1,cv2.LINE_4)
                        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),1)
                    elif respuesta==1:
                        titleobj = 'Car'
                        cv2.putText(frame, titleobj, (x,y-10),1,0.7,(0,255,0),1,cv2.LINE_4)
                        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),1)
                    elif respuesta==2:
                        titleobj = 'Articulated'
                        cv2.putText(frame, titleobj, (x,y-10),1,0.7,(0,255,0),1,cv2.LINE_4)
                        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),1)
                    elif respuesta==3:
                        titleobj = 'Bus'
                        cv2.putText(frame, titleobj, (x,y-10),1,0.7,(0,255,0),1,cv2.LINE_4)
                        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),1)
                    #elif respuesta==4:
                        #titleobj = 'Persona'
                        #cv2.putText(frame, titleobj, (x,y-10),1,0.7,(0,255,0),1,cv2.LINE_4)
                        #cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),1)
            
            
            #COUNT
            if 1186 < ( x + w) < 1189:
                car_C = car_C + 1
                cv2.line(frame, (1180,390),(1200,500),(0,255,0), 1)       
    
    
    cv2.drawContours(frame, [area_entrada], -1, color,2)
    cv2.rectangle(frame, (1230,40), (1278,0), (0,255,0),2)
    cv2.putText(frame, str(car_C), (1230,35), cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,255,0),2)
    cv2.drawContours(frame, [area_park], -1, color,2)
    cv2.putText(frame, Text, (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL,2,color,2)
    
    
    
    
    
    
    cv2.imshow('frame', frame)
    #cv2.imshow('GG', roi)
    k = cv2.waitKey(5) & 0xFF
    if k==27:
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
#DIMENSIONES
cap = cv2.VideoCapture('dia1.webm')
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print(height, width)

664.0 1278.0
